# 📦 Instalación de Librerías Necesarias para el Proyecto

Este proyecto requiere varias bibliotecas para el análisis y procesamiento de datos e imágenes. A continuación, se listan y describen:

- **pandas**: Manipulación y análisis de datos estructurados (tablas tipo DataFrame).
- **numpy**: Operaciones matemáticas avanzadas y manejo de arreglos multidimensionales.
- **matplotlib**: Visualización básica de gráficos e imágenes.
- **seaborn**: Visualización estadística avanzada basada en matplotlib.
- **opencv-python (cv2)**: Procesamiento de imágenes y visión por computadora.
- **scikit-image**: Lectura, filtrado y análisis de imágenes.
- **plotly**: Visualización interactiva y dinámica de gráficos.
- **kaggle**: Interfaz para descargar datasets desde [Kaggle.com](https://www.kaggle.com).

### ⏱ Tiempo estimado de instalación:
- Conexión buena: 3 a 7 minutos.
- Conexión lenta o Colab recién iniciado: 10 a 15 minutos.

### 💡 Recomendación:
- Si no va a trabajar directamente con imágenes, puede comentar o eliminar las siguientes librerías:
  - `opencv-python`
  - `scikit-image`

### ⚠️ Importante:
- La librería `kaggle` es necesaria solo si va a descargar datasets directamente desde Kaggle.
- Asegúrese de haber subido el archivo `kaggle.json` al entorno antes de autenticar con `KaggleApi`.

---


In [ ]:

!pip install -q kaggle
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install opencv-python
!pip install scikit-image
!pip install plotly


### 📥 Importación de librerías necesarias

Este bloque importa todas las librerías esenciales que se usarán a lo largo del proyecto. **Puede tardar alrededor de un minuto** en completarse, dependiendo del entorno y los recursos de tu máquina, ya que algunas son pesadas.

A continuación se explica el propósito de cada librería:

- **`os`**  
  Permite interactuar con el sistema operativo. Aquí se usa para cambiar la ubicación en la que `kaggle.json` será buscado (en este caso, la raíz del proyecto).

- **`kaggle.api.kaggle_api_extended.KaggleApi`**  
  Se utiliza para autenticar y descargar datasets directamente desde [Kaggle](https://www.kaggle.com).

- **`pandas` (`pd`)**  
  Manejo y análisis de datos en estructuras tipo tabla (`DataFrame`).

- **`numpy` (`np`)**  
  Cálculo numérico y manejo eficiente de arreglos multidimensionales.

- **`shutil`**  
  Permite copiar, mover o eliminar archivos y carpetas; útil para organizar las imágenes.

- **`cv2` (OpenCV)**  
  Librería de visión por computadora para leer, modificar y procesar imágenes.

- **`matplotlib.pyplot` (`plt`)**  
  Visualización de datos mediante gráficos estáticos y personalizables.

- **`seaborn` (`sns`)**  
  Complemento de `matplotlib` para gráficos estadísticos más atractivos y fáciles de generar.

- **`plotly.graph_objs` (`go`)**  
  Generación de gráficos interactivos (por ejemplo, líneas, barras, mapas de calor).

- **`skimage.io`**  
  Lectura y escritura de imágenes desde archivos o URLs, útil para visualización y análisis.


In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()#cambio de la ubicacion de busqueda de el kaggle.json para buscarlo en la raiz del proyecto
from kaggle.api.kaggle_api_extended import KaggleApi

import pandas as pd
import numpy as np
import shutil 
import cv2 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
from skimage import io


In [ ]:
directorio = "zargazo_dataset"

# Verifica si existe la carpeta para el data set del zargazo, si no la crea
if not os.path.exists(directorio):
    os.makedirs(directorio)
    print(f"✅ Carpeta creada: {directorio}")
else:
    print(f"ℹ️ La carpeta ya existe: {directorio}")

In [ ]:
#verifica si el data set ya fue descargado
if os.path.exists(directorio) and len(os.listdir(directorio)) > 0:
    print(f"✅ Dataset ya descargado en ./{directorio}.")
else:
    print("📥 Dataset no encontrado, descargando...")
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files("irvingvasquez/publicsargazods", path=directorio, unzip=True)
    print(f"✅ Dataset descargado en ./{directorio}")

In [ ]:
ruta_labels = os.path.join(directorio, "labels", "labels.csv")
df_train = pd.read_csv(ruta_labels)
datos = df_train.iloc[:,:] # Seleccionar un subconjunto
bd_lenght, bd_width = datos.shape
datos.sample(5)

In [ ]:
# imprimir cuales son las etiquetas únicas
print(datos.label.unique())

In [ ]:
datos.info()
print('Tipo de datos de las etiquetas: ', type(datos['label'][0]))

In [ ]:
datos.describe()

In [ ]:
#graficos de distriibucion de nivel de sargazo

plt.figure(figsize=(8,4))
sns.countplot(x='label',data=datos)

In [ ]:
#graficos de distribucion de las etiquetas de las fotos

plt.figure(figsize=(8,4))
sns.countplot(x='scene',data=datos)

In [ ]:
# Imágenes de ejemplo

import random

idx = random.randint(1, bd_lenght)
imagen_ejemplo=io.imread(directorio +"/images/"+ str(datos.image_name.iloc[idx]))

plt.title("Sample image, class=" + str(datos.label.iloc[idx]))
plt.imshow(imagen_ejemplo,vmin=0,vmax=1)

In [ ]:
# Para obtener información de la imagen podemos utilizar dtype y shape 
print('La imagen es de tipo:', imagen_ejemplo.dtype)
print("Dimensiones de la imagen (high, width, channels):", imagen_ejemplo.shape)

In [ ]:
#instalacion de librerias para red neuronal

!pip install scikit-learn
!pip install torch torchvision



In [ ]:
#librerias para la red neuronal

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
DATASET_PATH = os.path.join("zargazo_dataset","images")
CSV_PATH = os.path.join("zargazo_dataset", "labels", "labels.csv")
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 15

In [ ]:
df = pd.read_csv(CSV_PATH)

# Usar solo columnas necesarias
df = df[['image_name', 'label']]

# Mapear etiquetas a números
labels = df['label'].unique()
label2idx = {label: idx for idx, label in enumerate(labels)}
idx2label = {v: k for k, v in label2idx.items()}
df['label_idx'] = df['label'].map(label2idx)


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_idx'], random_state=42)


In [ ]:
class SargazoDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.loc[idx, 'image_name'])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.df.loc[idx, 'label_idx']
        return image, label


In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

train_dataset = SargazoDataset(train_df, DATASET_PATH, transform)
val_dataset = SargazoDataset(val_df, DATASET_PATH, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * (IMG_SIZE//8) * (IMG_SIZE//8), 128), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
#empezar entrenamiento

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN(num_classes=len(label2idx)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
#evaluacion

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

target_names = [idx2label[i] for i in sorted(idx2label.keys())]
print(classification_report(y_true, y_pred, target_names=target_names))



In [ ]:
CHECKPOINT = "modelo_sargazo.pth"
torch.save(model.state_dict(), CHECKPOINT)

In [ ]:
import shutil
from pathlib import Path

# Ruta destino, por ejemplo: Drive o carpeta personalizada
CHECKPOINT_DRIVE_DIR = Path("checkpoints_guardados")  # Cambia esto según tu caso

# Crear carpeta si no existe
CHECKPOINT_DRIVE_DIR.mkdir(parents=True, exist_ok=True)

# Copiar archivo
destination_path = CHECKPOINT_DRIVE_DIR / CHECKPOINT
shutil.copy(CHECKPOINT, destination_path)

print(f"Checkpoint copied to {destination_path}")

In [ ]:
model.load_state_dict(torch.load(destination_path))
model.to(device)
model.eval()


In [ ]:
# ------------------------- TEST EVALUATION ----------------------

# Asegurarse de que el modelo está en modo evaluación
model.eval()

all_preds = []
all_labels = []
all_images = []

with torch.no_grad():
    for x_batch, y_batch in val_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass (logits)
        logits = model(x_batch)

        # Predicciones usando argmax
        preds = torch.argmax(logits, dim=1)

        # Guardar resultados en CPU
        all_preds.append(preds.cpu())
        all_labels.append(y_batch.cpu())
        all_images.append(x_batch.cpu())

# Concatenar todos los batches
all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()
all_images = torch.cat(all_images)

# Evaluar desempeño
matches = all_preds == all_labels
right_preds = np.sum(matches)
wrong_preds = len(all_labels) - right_preds

# Mostrar métricas
print(f"\n✅ Test set results:")
print(f"  - Predicciones correctas: {right_preds} de {len(all_labels)}")
print(f"  - Predicciones incorrectas: {wrong_preds}")
print(f"  - Accuracy: {right_preds / len(all_labels):.2%}")
